# We have three files: one in .txt, one in .xlsm and one, optional, in .xlsx
# Firstly let's rework .txt file

In [1]:
# importing modules
import shutil  # for file manipulation
import os
import win32com.client as win32
from ok_sap_script import *
import time
from datetime import date, timedelta, datetime, timezone
import pyautogui
import pandas as pd
import numpy as np
from functools import wraps
import logging

# This is to work on SAP part
# S_PL0_86000030
class Control():
    def __init__(self):

        # Main variables
        self.curr_user = os.getlogin()
        self.date_stamp = datetime.today().strftime("%d-%m-%Y")
        self.receiver = "oleksandr.komarov@zoetis.com"
        # Logging
        self.log_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GIT/GIT_log_for_{self.date_stamp}.txt"
        self.main_logger = MyLogger(self.log_file_path)
        self.main_logger.log(level=logging.INFO, message="Test message")
        
        # SAP variables global
        self.environment = "ECC Production"
        
        # SAP variables S_PL0_xxx
        self.t_code_sap = "S_PL0_86000030"
        self.variant_sap = "GIT_AUTO"
        self.period = date.today().strftime("%#m") # use # to remove a leading 0
        self.period_0 = date.today().strftime("%m")
        self.year = datetime.today().year # get current year

        # SAP variables spool
        self.t_code_sap_spool = "sm37"
        self.job_name = "*"
        self.sap_person_name = "*"
        self.abap_prog_name = "ZR2RI_SIT_OUTBOUND"

        self.extract_path_spool = fr"C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\GIT\\Output\\"
        self.extract_name_spool = f"spool_GIT_for_{self.date_stamp}.txt"

        # SAP variables faglb03
        self.variant_sap_faglb03 = "GIT_AUTO"

        # Paths to files
            # model file
        self.model_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GIT/GIT_model_file.xlsb"
        self.new_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GIT/Output/GIT_for_{self.date_stamp}.xlsb"
            # spool file path
        self.spool_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GIT/Output/spool_GIT_for_{self.date_stamp}.txt"
            # S_PL0_86000030 file path
        self.spl0_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GIT/Output/SP_report_for_{self.date_stamp}"
            # historical cost file
        self.final_path_to_historical_cost = f"S:/Robotics_COE_Prod/RPA/MM60/Outputs/{self.year}/{self.period_0}/"
        self.historical_cost_file = f"Historical cost AP{self.period_0} LE2941.xlsm"
        self.final_joined_path_to_historical_cost = f"{self.final_path_to_historical_cost}{self.historical_cost_file}"
                # same but on my local drive
        self.new_file_path_to_historical_cost = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GIT/Output/{self.historical_cost_file}"
            # screenshots
        self.spl_scr = f"spl_scr_for_{self.date_stamp}"
        self.faglb03_scr = f"faglb03_scr_for_{self.date_stamp}"
        self.spool_scr_1 = f"spool_1_scr_for_{self.date_stamp}"
        self.spool_scr_2 = f"spool_2_scr_for_{self.date_stamp}"
            # list them
        self.list_screenshots = [self.spl_scr, self.faglb03_scr, self.spool_scr_1]
            # path to screenshots
        self.picture_path = fr'C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\GIT\\Screenshots\\'

    def __call__(self, *args, **kwargs):
        #self.run_logic_sap_spool() # this function will extract spool from SAP
        #self.run_logic_sap_SP() # this function will perform S_PL0_86000030 part, extracting the file before saving it with a timestamp
        self.sap_faglb03("faglb03")
        #self.run_logic_HC() # extract historical cost file from PRA folder 
        #self.create_final_file()
        x = 1+1

    def sap_decorator(sap_function):
        '''Closing SAP sessions before and after running a script'''
        @wraps(sap_function)
        def sap_wrapper(self, t_code_sap, *args, **kwargs):
            sap_close() # ensure there are no open session
            self.main_logger.log(level=logging.INFO, message="SAP is terminated")
            open_sap = sap_open() # open new session
            self.main_logger.log(level=logging.INFO, message="New SAP session is opened")
            time.sleep(5) # make sure SAP opens up
            if open_sap == True:
                self.main_logger.log(level=logging.INFO, message=f"Trying to enter {self.environment} environment")
                self.session = sap_logon(environment=self.environment, client=1)
                self.main_logger.log(level=logging.INFO, message=f"Trying to execute {t_code_sap} code in SAP")
                sap_code(tcode=t_code_sap, session=self.session)
                sap_function(self) # this is the main fuction to be decorated
            else:
                self.main_logger.log(level=logging.ERROR, message="SAP did not open")
            sap_close() # close SAP
            self.main_logger.log(level=logging.INFO, message="SAP is terminated")
        return sap_wrapper

    def timer_decorator(func_to_time):
        '''This one is used to track the execution time of any particular function'''
        import time
        @wraps(func_to_time)
        def time_wrapper(self, *args, **kwargs):
            t1 = time.time()
            func_to_time(self, *args, **kwargs)
            t2 = time.time() - t1
            print('{} run in: {} seconds'.format(func_to_time.__name__, round(t2,2)))
        return time_wrapper

    def create_final_file(self):
        #--Reworking spool file:
        # looks like a very nasty .txt file, which requires specific approach
        self.df_sap = pd.read_csv(self.spool_file_path, 
                            on_bad_lines='skip', sep="\t", encoding="ANSI",skiprows=13, skipinitialspace = True)
        self.df_sap = self.df_sap.loc[:, ~self.df_sap.columns.str.contains('^Unnamed')] # drop all unnamed columns -> ~ stands for bool
        self.df_sap = self.df_sap[self.df_sap['Plant'] != "Plant"] # drop all rows that have "Plant" in their names (those are repetitions of headers)
        self.df_sap.dropna(subset=['Plant'], inplace=True)
        # convert respective columns to numeric
        self.df_sap['Quantity'] = self.df_sap['Quantity'].str.replace(',', '').astype(float)
        self.df_sap['Amount in LC'] = self.df_sap['Amount in LC'].str.replace(',', '').astype(float)
        self.df_sap['Net Order Value in PO Curr.'] = self.df_sap['Net Order Value in PO Curr.'].str.replace(',', '').astype(float)
        self.df_sap['PO Quantity'] = self.df_sap['PO Quantity'].str.replace(',', '').astype(float)

        # export to excel
        self.git_tab_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GIT/Output/GIT_tab_for_{self.date_stamp}.xlsx"
        self.df_sap.to_excel(self.git_tab_path, index=False)

        #--Data Reworked APxx-xx tab:
        self.df_sap_le = self.df_sap.copy() # creating a copy of the file with only 4 LEs 
        self.df_sap_le = self.df_sap_le[(self.df_sap_le['Company Code'] == '2941') | (self.df_sap_le['Company Code'] == '2942') | 
                            (self.df_sap_le['Company Code'] == '2946') | (self.df_sap_le['Company Code'] == '2951')]
        # reset index:
        self.df_sap_le = self.df_sap_le.reset_index(drop=True)
        # add index column:
        self.df_sap_le['INDEX'] = self.df_sap_le.index + 2
        # adding columns:
        # concatenate:
        self.df_sap_le.insert(loc=0, column='Concatenate', value=self.df_sap_le['Material Number'] + "-" + self.df_sap_le['Plant'])
        # other:
        self.df_sap_le['BUoM historic'] = "=VLOOKUP(A" + self.df_sap_le['INDEX'].astype(str) + ",MM60!A:I,7,FALSE)"
        print(self.df_sap_le['BUoM historic'])
        self.df_sap_le['Std price per BUoM'] = "=VLOOKUP(A" + self.df_sap_le['INDEX'].astype(str) + ",MM60!A:I,9,FALSE)"
        self.df_sap_le['Value at historical cost'] = "=IF(AJ" + self.df_sap_le['INDEX'].astype(str) + "=AC" + self.df_sap_le['INDEX'].astype(str) + ",0,D" + self.df_sap_le['INDEX'].astype(str) + "*AO" + + self.df_sap_le['INDEX'].astype(str) + ")"
        self.df_sap_le['Test UOM'] = "=AN" + self.df_sap_le['INDEX'].astype(str) + "=E" + self.df_sap_le['INDEX'].astype(str)
        self.df_sap_le['Diff $'] = "=IF(AJ" + self.df_sap_le['INDEX'].astype(str) + "=AC" + self.df_sap_le['INDEX'].astype(str) + ",0,AP" + self.df_sap_le['INDEX'].astype(str) + "-F" + self.df_sap_le['INDEX'].astype(str) + ")"
        # drop index colums:
        self.df_sap_le = self.df_sap_le.drop('INDEX', axis=1)
        # export to excel:
        self.reworked_tab_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GIT/Output/Reworked_tab_for_{self.date_stamp}.xlsx"
        self.df_sap_le.to_excel(self.reworked_tab_path, index=False) 
    # working with RPA mm60 file
        try:
            self.df_mm60 = pd.read_excel(self.new_file_path_to_historical_cost, 
                        sheet_name="MM60 Report", converters={'Price':float})
            pd.options.display.float_format = '{:20,.2f}'.format # handling scientific notation
            # add columns:
            self.df_mm60.insert(loc=0, column='Concatenate', value=self.df_mm60['Material'] + "-" + self.df_mm60['Plant'])
            self.df_mm60['Std price per unit'] = self.df_mm60['Price'] / self.df_mm60['Price unit']
            self.mm60_tab_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GIT/Output/mm60_tab_for_{self.date_stamp}.xlsx"
            self.df_mm60.to_excel(self.mm60_tab_path, index=False)
        except Exception as e:
            print(f"Something went wring with converting the historical file, namely {e}")

            '''# Working with Roel file
            # df_roel = pd.read_excel(roel_file,
            #             converters={'BusA': str})
            # # remove all unwanted columns:
            # df_roel = df_roel[['Plnt', 'Material', 'BusA', 'Standard price', 'Crcy', 'BUn', 'per']]
            # # add useful columns:
            # df_roel.insert(loc=0, column='Concatenate', value=df_roel['Material'] + "-" + df_roel['Plnt'])
            # df_roel['Std price per unit'] = df_roel['Standard price'] / df_roel['per']
            # # drop last row:
            # df_roel.drop(df_roel.tail(1).index,inplace=True) # drop last (n) rows
            # mm60_tab_path = f"C:/Users/{curr_user}/Desktop/GIT_BOT/Workings/mm60_tab.xlsx"
            # df_roel.to_excel(mm60_tab_path, index=False)'''
# activating Excel:
        self.excel = win32.Dispatch("Excel.Application")
        self.excel.AskToUpdateLinks = False
        self.excel.DisplayAlerts = False
        self.excel.Visible = True 
        self.excel.ScreenUpdating = False
#--Active sheet on temporary extract:
        self.git_tab_wb = self.excel.Workbooks.Open(self.git_tab_path)
        self.git_tab_ws = self.git_tab_wb.Sheets("Sheet1") 
        # new file
        self.output_wb = self.excel.Workbooks.Open(self.new_file_path) # open the new file to paste all the data there
        self.output_ws = self.output_wb.Sheets("GIT") # start from GIT sheet
        self.output_ws.Range("A:AL").Delete() # clean the space
        self.git_tab_ws.Range('A1').CurrentRegion.Copy(Destination = self.output_ws.Range('A1')) # paste new info
        self.output_ws.Cells.Replace("=", "=") # replace = with = to make formulas work
        self.git_tab_wb.Close() # close git_tab excel
# working with mm60 tab
        self.mm60_tab_wb = self.excel.Workbooks.Open(self.mm60_tab_path) # which file to open
        self.mm60_tab_ws = self.mm60_tab_wb.Sheets("Sheet1") # which sheet to open
        self.output_ws = self.output_wb.Sheets("MM60") # open sheet to paste the data
        self.output_ws.Range("A:I").Delete() # clean the space
        self.mm60_tab_ws.Range('A1').CurrentRegion.Copy(Destination = self.output_ws.Range('A1')) # paste the data
        self.output_ws.Cells.Replace("=", "=") # this one is for formulas to work
        self.mm60_tab_wb.Close() # close wb
# working with spl tab S_PL0_86000030
        self.spl_tab_wb = self.excel.Workbooks.Open(self.spl0_file_path) # which file to open
        self.spl_tab_ws = self.spl_tab_wb.Sheets("Sheet1")  # which sheet to open
        self.output_ws = self.output_wb.Sheets("S_PL0_86000030") # open sheet to paste the data
        self.output_ws.Range("A2:AN100000").Delete() # clean the space, spare 1st row (headers)
        self.spl_tab_ws.Range('A3:AN100000').Copy(Destination = self.output_ws.Range('A2')) # paste the data (from A3 because of the headers)
        self.spl_tab_wb.Close() # close wb
# working with reworked tab
        self.reworked_tab_wb = self.excel.Workbooks.Open(self.reworked_tab_path) # which file to open
        self.reworked_tab_ws = self.reworked_tab_wb.Sheets("Sheet1")  # which sheet to open
        self.output_ws = self.output_wb.Sheets("Reworked") # open sheet to paste the data
        self.output_ws.Range("A2:AR100000").Delete() # clean the space
        self.reworked_tab_ws.Range('A2:AR100000').Copy(Destination = self.output_ws.Range('A2')) # paste the new data
        self.output_ws.Cells.Replace("=", "=") # this one is for formulas to work
        self.reworked_tab_wb.Close() # close reworked wb, for it's no longer needed
# pasting screenhosts
        self.screenshots_ws = self.output_wb.Sheets("Screenshots")
        print(self.screenshots_ws)
        print(self.list_screenshots)
        self.add_screenshots(worksheet=self.screenshots_ws, list_screenshots=self.list_screenshots)
# final steps
        self.output_wb.RefreshAll() # refesh all the pivots we have in the workbook
        self.excel.ScreenUpdating = True
        self.output_wb.Close(True) # True is for saving the file
        close_excel()
        self.send_email() 
        
    def send_email(self):
        # send by email
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = self.receiver
            mail.Subject = f"Automatic email for price GIT control for {self.date_stamp}"
            mail.Body = f"Please consult the attachment"
            mail.Attachments.Add(f"{self.new_file_path}")
            #mail.Display(True)
            mail.Send()

    @timer_decorator
    def run_logic_HC(self): # extract historical cost from the shared drive (RPA hc)
        # copy model file
        self.copy_file(self.model_file_path, self.new_file_path)
        # copying hc file
        self.copy_file(self.final_joined_path_to_historical_cost, self.new_file_path_to_historical_cost)

    def run_logic_sap_spool(self):
        # ensure there are no open sessions
        sap_close()
        # open new session
        open_sap = sap_open()
        time.sleep(5)
        if open_sap == True:
            session = sap_logon(environment=self.environment, client=1)
            sap_code(tcode=self.t_code_sap_spool, session=session)
            sap_enter_spool(session, job_name=self.job_name, sap_user_name=self.sap_person_name, abap_prog_name=self.abap_prog_name)
            self.take_screenshot(name=self.spool_scr_1)
            sap_extract_txt(session=session, extr_path=self.extract_path_spool, extr_name=self.extract_name_spool)
            # close sap after running the code
            sap_close()

    def run_logic_sap_SP(self):
        # ensure there are no open sessions
        sap_close()
        # open new session
        open_sap = sap_open()
        time.sleep(5)
        if open_sap == True:
            session = sap_logon(environment=self.environment, client=1)
            sap_code(tcode=self.t_code_sap, session=session)
            try:
                sap_variant(session=session, var_to_use=self.variant_sap)
            except: 
                sap_variant(session=session, var_to_use=self.variant_sap, version=2) # in our case, V2 is the most probable scenario
            # adjust parameters
            session.findById("wnd[0]/usr/ctxtPAR_01").text = self.period
            session.findById("wnd[0]/usr/ctxtPAR_02").text = self.period
            session.findById("wnd[0]/usr/ctxtPAR_06").text = self.year
            # run the report
            sap_run(session=session)
            # see if whether it was saved before
            try:
                session.findById("wnd[1]/usr/sub:SAPLKEC1:0110/radCEC01-CHOICE[1,0]").select()
                session.findById("wnd[1]/tbar[0]/btn[0]").press()
            except: pass
            # extract the data
            session.findById("wnd[0]/usr/lbl[1,1]").setFocus()
            self.take_screenshot(name=self.spl_scr)
            session.findById("wnd[0]").sendVKey(2)
            session.findById("wnd[0]/usr/lbl[1,11]").setFocus()
            session.findById("wnd[0]").sendVKey(2)
            session.findById("wnd[0]/tbar[1]/btn[47]").press()
            session.findById("wnd[1]/usr/btnD2000_PUSH_01").press()
            session.findById("wnd[1]/tbar[0]/btn[0]").press()
            session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[0,0]").select()
            session.findById("wnd[1]/tbar[0]/btn[0]").press()
            session.findById("wnd[1]/tbar[0]/btn[0]").press()
            # run the logic to save Excel file generated
            self.save_extract_sap()
            sap_close() # close SAP
    
    def save_extract_sap(self):
        self.target_workbook_name = "Worksheet in Basis (1)"
        workbook = self.get_workbook_by_name()

        if workbook: # if True is returned
            print(f"Workbook {workbook.Name} is found")
            # save this workbook
            workbook.SaveAs(Filename=f"C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\GIT\\Output\\SP_report_for_{self.date_stamp}")
            # close excel
            self.excel.DisplayAlerts = True
            close_excel()
            # close sap
            sap_close()
        else: print(f"No workbook named '{self.target_workbook_name}' found.")

    def get_workbook_by_name(self):
        try:
            self.excel = win32.GetActiveObject("Excel.Application")
            self.workbooks = self.excel.Workbooks
            self.excel.DisplayAlerts = False

            for workbook in self.workbooks:
                print(workbook.Name)
                if workbook.Name == self.target_workbook_name: return workbook
            return None
        except Exception as e:
            print("Excel not found:", e)
            return None

    @timer_decorator
    @sap_decorator
    def sap_faglb03(self):
        # try to apply variant
        self.main_logger.log(level=logging.INFO, message=f"Applying {self.variant_sap_faglb03} variant in SAP")
        try:
            sap_variant(session=self.session, var_to_use=self.variant_sap_faglb03)
        except: 
            sap_variant(session=self.session, var_to_use=self.variant_sap_faglb03, version=2) # in our case, V2 is the most probable scenario
        # adjust parameters
        self.session.findById("wnd[0]/usr/txtRYEAR").text = self.year
        # run the report
        sap_run(session=self.session)
        self.main_logger.log(level=logging.INFO, message="Report is executed")
        # take balance
        balance_value = self.session.findById("wnd[0]/usr/cntlFDBL_BALANCE_CONTAINER/shellcont/shell").GetCellValue(f"{self.period}", "BALANCE")
        self.take_screenshot(name=self.faglb03_scr) # take screenshot of faglb03
        self.session.findById("wnd[0]/usr/cntlFDBL_BALANCE_CONTAINER/shellcont/shell").setCurrentCell(f"{self.period}","BALANCE")

    def copy_file(self, file_from, file_to):
        try:
            shutil.copy(file_from, file_to)
        except PermissionError as e:
            print("The file may be opened, could not create new file")
            os.system("taskkill /f /im  excel.exe")

    def take_screenshot(self, name):
        try: # try taking screenshot
            screenshot = pyautogui.screenshot()
            screenshot.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GIT/Screenshots/{name}.png')
            self.main_logger.log(level=logging.INFO, message=f"Screenshot is taken with a name of {name}")
        except Exception as e: 
            self.main_logger.log(level=logging.ERROR, message=f"Failed to take screenshot {name} for a reason {e}")

    def add_screenshots(self, worksheet, list_screenshots):
        try:
            # remove old screenshots
            self.pictures = worksheet.Pictures()
            for pic in self.pictures:
                pic.Delete()
            # add screenshots
            self.left, self.top, self.width, self.height = 0, 30, 792, 534
            for screenshot in list_screenshots:
                self.picture_filename = screenshot
                self.full_picture_path = f"{self.picture_path}{self.picture_filename}.png"
                # insert a new screenshot with given parameters
                print(self.full_picture_path, "this is path")
                self.picture = worksheet.Shapes.AddPicture(self.full_picture_path, LinkToFile=False, SaveWithDocument=True, Left=self.left, Top=self.top, Width=self.width, Height=self.height)
                self.top += self.height
        except Exception as e:
            print(f"No screenshot or {e}")



class MyLogger:

    # Define custom log levels
    Custom1 = 15 # number are for levels having different number behind (10 for Debug and 50 for Error, I suppose)
    Custom2 = 45

    def __init__(self, log_file_path, log_level=logging.INFO):
        '''Define path, define minimal log level, call setup function'''
        self.receiver_log = "oleksandr.komarov@zoetis.com" # my name in case of log failure
        self.log_file_path = log_file_path
        self.log_level = log_level
        self.logger = self._setup_logger()

    def _setup_logger(self): # _ means that this method is to be used inside this class only (even though nothing prevants you from using it anyway)
        logger = logging.getLogger(self.log_file_path)
        logger.setLevel(self.log_level)
        self._add_file_handler(logger) # how to handle log .txt file
        self._add_console_handler(logger) # how logs are displayed in the console

        return logger

    def _add_file_handler(self, logger):
        file_formatter = self._get_file_formatter()
        with open(self.log_file_path, mode='w') as log_file:
            file_handler = logging.FileHandler(self.log_file_path, mode='w')
            file_handler.setFormatter(file_formatter)
            logger.addHandler(file_handler)

    def _add_console_handler(self, logger):
        console_formatter = self._get_console_formatter()
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(console_formatter)
        logger.addHandler(console_handler)

    def _get_file_formatter(self):
        if self.log_level == logging.INFO:
            return logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
        else:
            return logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

    def _get_console_formatter(self):
        if self.log_level <= logging.INFO:
            return logging.Formatter('%(levelname)s - %(message)s')
        elif self.log_level <= logging.ERROR:
            return logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
        else:
            if self.log_level == MyLogger.Custom1:
                return logging.Formatter('%(levelname)s - %(message)s')
            elif self.log_level == MyLogger.Custom2:
                return logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
            else:
                return logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    def log(self, message, level=logging.INFO):
        '''Main function to write a log message'''
        self.logger.log(level, message)
        if level >= logging.ERROR:
            self.reason = message
            self._send_email_failed()

    def _send_email_failed(self):
        # send this failure by email
        self.outlook = win32.Dispatch('outlook.application')
        self.mail = self.outlook.CreateItem(0)
        self.mail.To = self.receiver_log
        self.mail.Subject = f"Control failure :("
        self.mail.Body = f"This is to inform you that one of the controls failed.\nReason {self.reason}\n.Please consult attached log file to know a reason."
        self.mail.Attachments.Add(f"{self.log_file_path}")
        #self.mail.Display(True)
        self.mail.Send()

if __name__ == "__main__": 
    control = Control()
    control()
    #control.faglb03_scr("faglb03")

INFO - Test message
INFO - SAP is terminated
INFO - New SAP session is opened
INFO - Trying to enter ECC Production environment
INFO - Trying to execute faglb03 code in SAP
INFO - Applying GIT_AUTO variant in SAP
INFO - Report is executed
INFO - Screenshot is taken with a name of faglb03_scr_for_29-08-2023
INFO - SAP is terminated


sap_faglb03 run in: 14.76 seconds


In [ ]:
import numpy as np
import pandas as pd

#--GIT APxx-xx tab:
# looks like a very nasty .txt file, which requires specific approach
df_sap = pd.read_csv("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/GIT_AP02.txt", 
                     on_bad_lines='skip', sep="\t", encoding="ANSI",skiprows=13, skipinitialspace = True)
df_sap = df_sap.loc[:, ~df_sap.columns.str.contains('^Unnamed')] # drop all unnamed columns -> ~ stands for bool
df_sap = df_sap[df_sap['Plant'] != "Plant"] # drop all rows that have "Plant" in their names (those are repetitions of headers)
df_sap.dropna(subset=['Plant'], inplace=True)
# convert respective columns to numeric
df_sap['Quantity'] = df_sap['Quantity'].str.replace(',', '').astype(float)
df_sap['Amount in LC'] = df_sap['Amount in LC'].str.replace(',', '').astype(float)
df_sap['Net Order Value in PO Curr.'] = df_sap['Net Order Value in PO Curr.'].str.replace(',', '').astype(float)
df_sap['PO Quantity'] = df_sap['PO Quantity'].str.replace(',', '').astype(float)
# export to excel
#df_sap.to_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/Pandas_GIT_AP02.xlsx", index=False)

#--Data Reworked APxx-xx tab:
df_sap_le = df_sap.copy() # creating a copy of the file with only 4 LEs 
df_sap_le = df_sap_le[(df_sap_le['Company Code'] == "2941") | (df_sap_le['Company Code'] == "2942") | 
                      (df_sap_le['Company Code'] == "2946") | (df_sap_le['Company Code'] == "2951")]
# reset index:
df_sap_le = df_sap_le.reset_index(drop=True)
# add index column:
df_sap_le['INDEX'] = df_sap_le.index + 2
# adding columns:
# concatenate:
df_sap_le.insert(loc=0, column='Concatenate', value=df_sap_le['Material Number'] + "-" + df_sap_le['Plant'])
# other:
df_sap_le['BUoM historic'] = "=VLOOKUP(A" + df_sap_le['INDEX'].astype(str) + ",historicalcostsv2,7,FALSE)"
df_sap_le['Std price per BUoM'] = "=VLOOKUP(A" + df_sap_le['INDEX'].astype(str) + ",historicalcostsv2,9,FALSE)"
df_sap_le['Value at historical cost'] = "=IF(AJ" + df_sap_le['INDEX'].astype(str) + "=AC" + df_sap_le['INDEX'].astype(str) + ",0,D" + df_sap_le['INDEX'].astype(str) + "*AO" + + df_sap_le['INDEX'].astype(str) + ")"
df_sap_le['Test UOM'] = "=AN" + df_sap_le['INDEX'].astype(str) + "=E" + df_sap_le['INDEX'].astype(str)
df_sap_le['Diff $'] = "=IF(AJ" + df_sap_le['INDEX'].astype(str) + "=AC" + df_sap_le['INDEX'].astype(str) + ",0,AP" + df_sap_le['INDEX'].astype(str) + "-F" + df_sap_le['INDEX'].astype(str) + ")"
print(df_sap_le)
# drop index colums:
df_sap_le = df_sap_le.drop('INDEX', axis=1)
# export to excel:
#df_sap_le.to_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/Pandas_GIT_Reworked.xlsx", index=False)


#print(df_sap_le)


# second part (work with mm60 extract)

In [ ]:
import numpy as np
import pandas as pd

df_mm60 = pd.read_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/Historical cost AP02 LE2941.xlsm", 
                        sheet_name="MM60 Report", converters={'Price':float})

pd.options.display.float_format = '{:20,.2f}'.format # handling scientific notation

# add columns:
#df_mm60['INDEX'] = df_mm60.index + 2
df_mm60.insert(loc=0, column='Concatenate', value=df_mm60['Material'] + "-" + df_mm60['Plant'])
df_mm60['Std price per unit'] = df_mm60['Price'] / df_mm60['Price unit']
print(df_mm60)
df_mm60.to_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/pandas_mm60_rpa.xlsx", index=False)

# if historical cost file is received from Roel

In [84]:
import numpy as np
import pandas as pd

df_roel = pd.read_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/historical costs AP 2 2023 - LE 2941.xlsx",
                        converters={'BusA': str})
# remove all unwanted columns:
df_roel = df_roel[['Plnt', 'Material', 'BusA', 'Standard price', 'Crcy', 'BUn', 'per']]
# add useful columns:
df_roel.insert(loc=0, column='Concatenate', value=df_roel['Material'] + "-" + df_roel['Plnt'])
df_roel['Std price per unit'] = df_roel['Standard price'] / df_roel['per']
# drop last row:
df_roel.drop(df_roel.tail(1).index,inplace=True) # drop last (n) rows

print(df_roel)

          Concatenate  Plnt  Material  BusA       Standard price  Crcy  BUn  \
0       10009694-AT00  AT00  10009694  9334                 6.33   NaN  SYR   
1       10020218-AT00  AT00  10020218  9334            19,562.83   NaN   PC   
2       10020219-AT00  AT00  10020219  9334               861.93   NaN   PC   
3       10020220-AT00  AT00  10020220  9334            17,135.68   NaN   PC   
4       10020221-AT00  AT00  10020221  9334             5,327.76   NaN   PC   
...               ...   ...       ...   ...                  ...   ...  ...   
107048  25001338-IE11  IE11  25001338  7246                 5.67   NaN   SU   
107049  25001339-IE11  IE11  25001339  7246                12.05   NaN   SU   
107050  25001340-IE11  IE11  25001340  7246                35.70   NaN   SU   
107051  25001341-IE11  IE11  25001341  7246                54.44   NaN   SU   
107052  25001342-IE11  IE11  25001342  7246                33.00   NaN   SU   

                        per   Std price per unit  
